# 14장 - 엔드투엔드 결절 분석, 그리고 남겨진 과제

In [ ]:
# 모든 구성 요소 결합해서 하나로 만들어 프로젝트의 최종 목표인 암 자동 탐지 완성

## 1절 - 결승선을 향하여

In [ ]:
# 세그멘테이션 모델과 분류기 연결하는 가교 생성해야 함
# 결절 분석 및 진단 해야 함

In [ ]:
# 중요 작업 세 가지
# 1. 결절 후보 생성: 전체 프로젝트 중 3 단계
#   a. 세그멘테이션: 세그멘테이션 모델은 2차원 단면에 대해 예측을 수행하며 이 결과를 쌓아 결절 후보 예측을 포함하는 3차원 복셀 배열 제작
#   b. 그룹화: 복셀에 기준값 적용해 결절 후보들로 그룹화하고, 이런 복셀의 연결된 영역을 그룹으로 묶기
#   c. 샘플 튜플 만들기: 식별된 결절 후보는 분류를 위한 샘플 튜플 만들기 위해 사용하는데, 특히 결절의 중심 좌표(인덱스, 행, 열) 생성
# 과정 마치면 환자의 원본 CT 스캔 입력 시 탐지 결과로 결절 후보 리스트 만드는 애플리케이션 완성

In [ ]:
# 2. 결절과 악성 종양 분류: 만들어진 결절 후보를 분류기에 통과시켜 후보에서 악성 골라냄
#   a. 결절 분류: 세그멘테이션과 그룹화에서 얻은 결절 후보는 결절 여부로 분류하여 해부학적으로 정상인 경우 버림
#   b. ROC/AUC 메트릭: 마지막 분류 단계 전에 분류 모델의 성능 가늠하기 위한 새로운 메트릭 정의하고 악성 분류 결과와 비교할 기준이 되는 메트릭 만들기
#   c. 악성 모델 미세 튜닝: 메트릭이 준비되면 양성과 악성 결절 구별하기 위한 모델 정의하고 훈련 후 성능 확인. 훈련은 미세 튜닝을 통해 진행되는데 기존 모델의 가중치 중 일부를 버리고 새 값으로 바꾸면서 분류 작업에 맞게 업데이트

In [ ]:
# 3. 엔드투엔드 탐지: 모든 것을 종합해 CT를 보고 악성 결절 여부를 답할 수 있는 엔드투엔드 솔루션 제작
#   a. IRC: CT를 세그멘테이션하여 분류할 결절 후보 샘플 얻기
#   b. 결절 여부 판단: 각 후보에 대해 결절 분류 수행해 악성 분류기에 넣어야 할지 결정
#   c. 악성 여부 판단: 환자에 암 있는지를 판단하는 분류기에 넣어 결저르이 악성 여부 판단

## 2절 - 검증셋 독립

In [ ]:
# 미묘하지만 심각한 실수 저지를 여지 있는 부분 방지 차원에서 언급할 내용임
# 훈련셋을 검증셋에 노출할 위험이 있음
# 세그멘테이션과 분류 모델에서 데이터를 훈련셋과 독립 검증셋으로 구분하기 위해 루프 돌며 열 번째 예제를 검증셋에 넣고 나머지를 훈련셋으로 함

In [ ]:
# 하지만 분류 모델을 위한 데이터 분리는 결절 리스트에 대해 진행했고 세그멘테이션 모델에 대한 분리는 CT 스캔 리스트에 대해 이뤄짐
# 따라서 세그멘테이션용 검증셋의 결절이 분류 모델을 위한 훈련셋에 들어가거나 반대의 상황 일어날 수 있음
# 고치지 않는다면 인위적으로 높은 성능 점수 얻도록 만드는 이른바 누출(leak) 상황임

In [ ]:
# 누출 가능성 없애기 위해 13장과 같이 분류용 데이터셋을 재처리하여 CT 스캔 레벨에서도 문제 없게 만들어야 함
# 이 새로운 데이터셋으로 분류 모델 재훈련하면 해결됨

In [ ]:
# 검증셋을 정의할 때는 엔드투엔드 프로세스에 주의를 기울여야 함
# 가장 쉬운 방법은 최대한 명백하게 검증 분리하는 것임
# 데이터 분리를 다시 하게 되면 새롭게 분리된 데이터셋으로 모든 모델 재훈련 해야 함

In [ ]:
# 분리 작업한 후에 재훈련 시킨 결과
# 100 에포크 지나자 양성 샘플 정확도 95%, 음성 샘플 정확도 99%
# 검증 손실이 증가하는 경향 보이지 않으므로 계속 훈련시켜도 됨
# 90 에포크에서 최대 F1 점수 도달, 실제 결절 정확도 92.8%, 검증셋에서의 정확도 99.2%
# 악성 결절에 대한 정확도 높이기 위해 전체 정확도 손해 보더라도 현 모델 사용

## 3절 - CT 세그멘테이션과 결절 후보 분류 연결

In [ ]:
# 저장된 세그멘테이션과 재훈련한 분류 모델 있으므로 그룹화 진행
# 입력은 세그멘테이션 모델을 통해 표시한 복셀 형태의 세그멘테이션
# 표시된 복셀 덩어리의 중심 좌표를 샘플 튜플 출력 리스트에 넣는 것이 목적

In [ ]:
# 모델 실행은 훈련 및 검증 단계에서 다룬 내용과 흡사
# 차이점은 CT를 도는 각 루프에서 모든 단면을 세그멘테이션한 결과를 그룹화 입력으로 사용
# 그룹화 결과는 결절 분류기로 들어가고 분류기에서 살아남은 결절은 악성 분류기에 들어감

In [ ]:
# 실제 구현은 CT 순회하는 루프에 존재
# 각 CT를 세그멘테이션하고, 그룹화한 다음, 후보 식별하고, 처리를 위한 분류기에 들어감
# nodule_analysis.py:324
for _, series_uid in series_iter: # 시리즈 uid 루프
    ct = getCt(series_uid) # CT 얻기
    mask_a = self.segmentCt(ct, series_uid) # 그 위에서 세그멘테이션 돌리기

    candidateInfo_list = self.groupSegmentationOutput( # 출력 중 표시된 복셀 그룹화
        series_uid, ct, mask_a)
    classifications_list = self.classifyCandidates( # 결절 분류기 돌리기
        ct, candidateInfo_list)

#### .1 세그멘테이션

In [ ]:
# 전체 CT 스캔의 모든 단면에 대해 세그멘테이션 수행
# 주어진 환자의 CT를 단면 단위로 입력할 수 있도록 CT와 하나의 series_uid 읽고 __getitem__ 호출 때마다 단면 하나씩 넘겨주는 Dataset 구현
# 다량의 입력 외에 주요 차이점은 출력

In [ ]:
# scipy.ndimage.morphology의 침식 연산(erosion operation)을 사용한 클린업 단계 삽입 가능
# 경계 복셀 지우고 내부만 유지하는 방식인데 축 방향에서 이웃하는 여덟 개의 복셀까지 표시된 경우 처리하기 위함
# 이 과정으로 플래그된 영역을 더 작게 만들어 아주 작은 구성 요소(3*3*3복셀보다 작은 경우)를 제거

In [ ]:
# CT 하나의 모든 단면 제공하는 데이터 로더있는 루프에 전체 다 넣어 구현
# nodule_analysis.py:384
def segmentCt(self, ct, series_uid):
    with torch.no_grad():
        output_a = np.zeros_like(ct.hu_a, dtype=np.float32) # 이 배열에 부동소수점 확률 배열 출력 들어감
        seg_dl = self.initSegmentationDl(series_uid)  #  배치에서 CT를 루프 도는 데이터 로더 얻기
        for input_t, _, _, slice_ndx_list in seg_dl:

            input_g = input_t.to(self.device) # GPU로 이동
            prediction_g = self.seg_model(input_g) # 세그멘테이션 모델 작동

            for i, slice_ndx in enumerate(slice_ndx_list): # 각 요소를 출력 배열로 복사
                output_a[slice_ndx] = prediction_g[i].cpu().numpy()

        mask_a = output_a > 0.5 # 이진값 출력을 만들기 위해 값에 경계값 적용하고 클린업 위해 이진 침식 연산 적용
        mask_a = morphology.binary_erosion(mask_a, iterations=1)

    return mask_a

#### .2 복셀을 그룹화하여 결절 후보로 만들기

In [ ]:
# 간단한 컴포넌트 연결 알고리즘으로 결절에 해당하는 것으로 추정되는 복셀들을 그룹화하여 분류기로 넘길 것임
# 이 그룹화 알고리즘 방법은 연결되는 컨포넌트를 레이블하는 방식으로 scipy.ndimage.measurements.label 사용
# label 함수는 동일한 경계선에 있는 0이 아닌 픽셀을 동일한 그룹에 포함되는 것으로 표시함
# 세그멘테이션 모델의 출력에 서로 상당히 인접하는 픽셀 덩어리가 포함되어 있으므로 효과적임
# nodule_analyze.py:401
def groupSegmentationOutput(self, series_uid,  ct, clean_a):
    candidateLabel_a, candidate_count = measurements.label(clean_a) # 각 복셀을 속하는 그룹으로 레이블링 함
    centerIrc_list = measurements.center_of_mass( # 각 그룹의 중심을 인덱스, 행, 열 좌표로 얻음
        ct.hu_a.clip(-1000, 1000) + 1001,
        labels=candidateLabel_a,
        index=np.arange(1, candidate_count+1),
    )

In [1]:
# 출력 배열 candidateLabel_a는 입력에 사용된 clean_a와 같은 차원임
# 다만 배경 복셀은 0이며 정수인 레이블 값은 1, 2, ...처럼 증가
# 각 숫자에 해당하는 그룹에는 결절 후보를 구성하는 인접한 복셀 덩어리 들어있음

In [ ]:
# scipy에는 결절 후보의 중심 구해주는 scipy.ndimage.measurements.center_of_mass 함수 존재
# 복셀 단위의 밀도값과 호출한 label 함수에서 얻은 정수 레이블 그리고 중심값 계산이 필요한 레이블 리스트 입력받음
# 입력으로 들어가는 밀도값은 음수가 될 수 없으므로 ct.hu_a에 오프셋을 1001 더함
# 공기에 해당하는 제일 작은 값이 -1000HU이므로 플래그가 켜진 복셀에 가중치 부여하게 되는 셈
# nodule_analyze.py:409
candidateInfo_list = []
for i, center_irc in enumerate(centerIrc_list):
    center_xyz = irc2xyz( # 복셀 좌표를 실제 환자 좌표로 변환
        center_irc,
        ct.origin_xyz,
        ct.vxSize_xyz,
        ct.direction_a,
    )
    assert np.all(np.isfinite(center_irc)), repr(['irc', center_irc, i, candidate_count])
    assert np.all(np.isfinite(center_xyz)), repr(['xyz', center_xyz])
    candidateInfo_tup = \
        CandidateInfoTuple(False, False, False, 0.0, series_uid, center_xyz) # 후보 정보 튜플을 만들어 탐지 리스트에 붙임
    candidateInfo_list.append(candidateInfo_tup)

return candidateInfo_list

In [2]:
# 출력으로 candidate_count와 길이가 동일한 배열 3개(인덱스, 행, 열)의 리스트 얻음
# 이 데이터는 candidateInfo_tup 인스턴스 리스트를 만들 때 사용
# 앞의 네 값(isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm) 넣을 곳이 없으므로 적당한 타입 공간 만들어 넣음
# 이후 복셀의 좌표를 루프에서 물리 좌표로 변환하고 리스트 생성
# candidatesInfo_tup 인스턴스를 사용하는 곳에서 center_xyz를 기대하므로 바꿔 사용하면 잘못된 결과가 출력됨

#### .3 결절을 찾았나? 거짓 양성을 줄이기 위한 분류

In [ ]:
# 1. 세그멘테이션: 2^25개 복셀에서 97% 버린 2^20 복셀 남음
# 2. 그룹화: 복셀을 결절 후보로 병합하는 과정에서 2^10 복셀로 줄어듦
# 3. 결절 분류: 2^5 정도로 줄어듦
# 4. 악성 분류: 2^1 개 정도의 암을 찾음

In [ ]:
# 세그멘테이션 모델로 문제 되는 후보 이미지 영역 구별하고 CT에서 각 후보들 떼어내어 분류 모듈에 입력
# candidateInfo_list로 DataSet을 만들고 DataLoader에 넣어 순회시키기
# 확률 예측의 1번 열은 우리가 유지하려는 결절 여부에 대한 확률값 예측
# nodule_analyze.py:357
def classifyCandidates(self, ct, candidateInfo_list):
    cls_dl = self.initClassificationDl(candidateInfo_list) # candidateInfo_list에 대해 루프 돌 데이터 로더 얻기
    classifications_list = []
    for batch_ndx, batch_tup in enumerate(cls_dl):
        input_t, _, _, series_list, center_list = batch_tup

        input_g = input_t.to(self.device)
        with torch.no_grad():
            _, probability_nodule_g = self.cls_model(input_g) # 입력을 결절vs비결절 신경망에 넣기
            if self.malignancy_model is not None: # 악성 모델 있으면 넣기
                _, probability_mal_g = self.malignancy_model(input_g)
            else:
                probability_mal_g = torch.zeros_like(probability_nodule_g)

        zip_iter = zip(center_list,
            probability_nodule_g[:,1].tolist(),
            probability_mal_g[:,1].tolist())
        for center_irc, prob_nodule, prob_mal in zip_iter: # 계산 수행해 결과 리스트 만들기
            center_xyz = irc2xyz(center_irc,
                direction_a=ct.direction_a,
                origin_xyz=ct.origin_xyz,
                vxSize_xyz=ct.vxSize_xyz,
            )
            cls_tup = (prob_nodule, prob_mal, center_xyz, center_irc)
            classifications_list.append(cls_tup)
    return classifications_list

In [ ]:
# 출력된 확률을 경계값 기준으로 걸러서 모델이 결절이라고 판단 내린 경우 모아 리스트로 출력
# nodule_analyze.py:333
    if not self.cli_args.run_validation:
        print(f"found nodule candidates in {series_uid}:")
        for prob, prob_mal, center_xyz, center_irc in classifications_list:
            if prob > 0.5:
                s = f"nodule prob {prob:.3f}, "
                if self.malignancy_model:
                    s += f"malignancy prob {prob_mal:.3f}, "
                s += f"center xyz {center_xyz}"
                print(s)

    if series_uid in candidateInfo_dict:
        one_confusion = match_and_score(
            classifications_list, candidateInfo_dict[series_uid]
        )
        all_confusion += one_confusion
        print_confusion(
            series_uid, one_confusion, self.malignancy_model is not None
        )

print_confusion(
    "Total", all_confusion, self.malignancy_model is not None
)

In [ ]:
# 검증셋에서 얻은 CT로 돌리자 16개의 결절 후보 찾음
# 검증셋을 사용하므로 각 CT에 대해 모든 애노테이션과 악성 정보 가짐
# 덕분에 결과로 혼동 행렬 만들 때 이런 정보 사용 가능
# 행이 애노테이션에서 정의한 참이며 열은 프로젝트에서 각 경우를 어떻게 다루는지 보여줌

In [ ]:
# Complete Miss열은 세그멘테이션 모델이 결절에 대해 전혀 표시하지 않은 경우
# 세그멘테이션이 비결절인 경우 표시하려 하지 않으므로 셀은 비움
# 세그멘테이션 모델은 높은 재현율 가지도록 훈련되었으므로 결절이 아닌 경우가 많지만 결절 분류기가 잘 동작해 충분히 걸러냄

In [ ]:
# 스캔에서 한 개의 악성 결절 찾았지만 17번째 양성 놓침
# 15개의 거짓 양성인 비결절의 경우는 결절 분류기로 들어감
# 분류기로 필터링한 결과 1000개 이상의 거짓 양성 줄임

## 4절 - 정량적인 검증

In [ ]:
# 전체 검증셋에 대한 모델 성능 확인
# 검증셋에 대해 앞서의 예측 돌려보고 얼마나 많은 결절 찾고 놓쳤는지, 얼마나 많은 후보를 결절로 잘못 판단했는지 확인

In [ ]:
# 154개 결절 중 132개, 85% 탐지함
# 놓친 22개 중 13개는 세그멘테이션에서 후보로 고려되지 않았으므로 개선이 필요
# 탐지한 결절 중 95%는 거짓 양성

## 5절 - 악성 예측

In [ ]:
# 악성 결절 구별은 환자에 대한 종합적인 진단, CT 외의 정보, 생체 검사가 수반되야 함

#### .1 악성 정보 습득

In [ ]:
# pylicd는 악성 정보를 제공함
# LIDC의 애노테이션 정보를 LUNA 후보의 좌표 정보와 연결할 필요 있음
# LIDC 애노테이션에는 결절별로 방사선 전문의가 악성 여부를 1~5로 표기한 정보 인코딩되어 있음
# 최소 두 명의 전문의가 4단계 이상으로 애노테이션 달아놓은 경우라면 해당 결절은 악성으로 간주
# mal_list와 ben_list에서 샘플링

In [ ]:
# dataset 명령행 인자 만들어 지정된 데이터셋 클래스를 동적으로 사용
# getattr 함수로 구현 가능
# training.py:154
ds_cls = getattr(p2ch14.dsets, self.cli_args.dataset) # 동적 클래스 네임 탐색

train_ds = ds_cls(
    val_stride=10,
    isValSet_bool=False,
    ratio_int=1, # 훈련 데이터 양성 및 악성 일대일 밸런
)

#### .2 AUC 베이스라인: 직경으로 분류하기

In [ ]:
# 성능 향상 확인을 위해 기준치를 설정
# 악성 진단을 위한 예측 도구로 직경 크기 사용 -> 크기가 큰 결절은 악성 가능성이 높다는 가정

In [ ]:
# 결절의 악성 여부 예측하기 위한 가상의 분류기에 결절의 직경을 유일한 입력으로 사용
# '경계값보다 큰 모든 결절은 악성'이라는 가정은 악성 여부 예측에 괜찮은 방법으로 알려져 있음
# 적당한 경계값을 찾는 것이 핵심
# 큰 종양 모두 포함하고 작은 돌기 하나도 없는 지점으로서, 크지만 양성인 결절과 작지만 악성인 결절이 함께 포진해 있는 불확실한 영역도 대략적으로 구분해 주는 경계값

In [ ]:
# 재현율을 생각하면 경계값 변경에 따라 결과가 바뀜
# 거짓 양성 비율(FPR)= FP / (FP + TN), 참 양성 비율(TPR)= TP / (TP + FN)

In [ ]:
# 결절 데이터에서 결절 크기는 3.25~22.78밀리 범위를 가짐
# FPR 값을 X, TPR 값을 Y로 놓으면 경계값 나타내는 그래프 그릴수 있고 ROC(receiver operating Characteristic) 다이어그램을 그릴 수 있음
# ROC 영역 아래는 AUC(are under curve)로 부르며 AUC는 0~1 값으로 높을수록 좋음
# 5.42밀리보다 작은 경우 TPR만 떨어지고 10.55밀리보다 크면 이득 없이 악성 결절을 양성으로 간주
# 따라서 최적의 경계값은 이 사이에 존재

In [ ]:
# 결절 정보 리스트 가져온 다음, 결절로 애노테이션된 것은 날리고 악성 레이블과 직경 얻음
# p2ch14_malben_baseline.ipynb
ds = p2ch14.dsets.MalignantLunaDataset(val_stride=10, isValSet_bool=True)  # 정규 데이터셋에서 양성과 악성 결절 리스트 얻기
nodules = ds.ben_list + ds.mal_list
is_mal = torch.tensor([n.isMal_bool for n in nodules])  # 악성 상태와 직경 리스트 얻기
diam  = torch.tensor([n.diameter_mm for n in nodules])
num_mal = is_mal.sum()  # TPR과 FPR 정규화를 위해 악성과 양성 결절 수 얻기
num_ben = len(is_mal) - num_mal

In [ ]:
# ROC 곡선 계산 위한 경계가 될 값의 배열이 필요
# 이를 위해 두 경계값을 입력 받는 torc.linspace 사용
# 양성 예측이 0인 경우부터 시작하도록 경계값이 최대에서 최소까지 진행하도록 만듦
threshold = torch.linspace(diam.max(),diam.min())

In [ ]:
# 행은 각 경계 나타내고 열은 샘플별 정보 의미하며 값은 각 샘플이 양성으로 예측되었는지 의미하는 2차원 텐서 제작
# 이 불리언 텐서는 샘플 레이블이 악성인지 아닌지 구별
# 행 방향으로 합쳐서 True 수 세기
# 이후 악성 결절 수나 양성 결절 수로 나누면 TPR과 FPR 얻을 수 있으므로 이를 ROC 곡선의 좌표로 사용
predictions = (diam[None] >= threshold[:, None]) # None으로 인덱싱하여 .unsqueeze(ndx)처럼 크기 1인 차원 넣어 열에 있는 결절이 행에 있는 직경을 기준으로 악성으로 분류됐는지에 대한 2차원 텐서 만들게 됨
tp_diam = (predictions & is_mal[None]).sum(1).float() / num_mal # 예측 행렬로 열 단위로 합쳐 얻은 각각의 직경에 대해 TPR과 FPR 계산
tp_diam = (predictions & ~is_mal[None]).sum(1).float() / num_ben

In [ ]:
# AUC 아래 면적을 계산하기 위해 사다리꼴 공식을 이용한 수치 적분 수행
# 두 점 사이의 Y축 TPR의 평균을 구한 후 X축 FPR의 최댓값과 최솟값의 차이를 구하여 곱하는 식으로, 그래프의 두 점 사이의 사다리꼴 영역의 면적을 구함
# 이후 이 값들을 다시 합함
fp_diam_diff = fp_diam[1:] - fp_diam[:-1]
tp_diam_avg = (tp_diam[1:] + tp_diam[:-1]) / 2
auc_diam = (fp_diam_diff * tp_diam_avg).sum()

#### .3 이전 가중치의 재활용: 미세 튜닝

In [ ]:
# 더 적은 데이터로 결과를 빨리 얻는 방법 중의 하나는 랜덤 초기화 대신 관련 데이터로 이미 훈련된 신경망에서 시작하는 것
# 미세 튜닝 혹은 전이 학습(trainsfer learning)이라고 부르는데 뒤 단의 몇 계층만 훈련하는 것을 의미
# 모델 뒷부분을 잘라 새로운 것으로 바꾼다고 생각하면 됨

In [ ]:
# 중간 과정에서 나온 값은 이미지에서 추출된 피처라고 볼 수 있음
# 이미지 추출 피처란 패턴 탐지 또는 모델이 감지하는 경계나 모서리 인식에 해당
# 미세 튜닝 작업은 이미 존재하는 피처를 사용하는 예전 방식과 학습을 통해 피처를 얻는 새로운 방식의 결합
# 신경망의 일부는 고정된 피처 추출기가 되고 그 위에서 상대적으로 작은 계층만 훈련

In [ ]:
# 일반적으로 잘 동작하지만 잘 되지 않는 경우도 있음
# 좌우 구별이 필요한 작업에 좌우반전 증강이 적용된 모델을 사용하면 정확도에 문제가 발생

In [1]:
# 비슷한 데이터에서 훈련된 신경망인 결절 분류 신경망 있으므로 사용
# 기본적인 미세 튜닝 방법 유지할 것임
# 랜덤 초기화는 그대로 둔 상태에서 head_linear 부분 잘라낸 후 head_linear와 마지막 컨볼루션 블럭 모두 재훈련시킴

In [2]:
# 다음 진행해야 함
# 시작할 때 사용할 (초기화 후 고정할) 모델의 가중치를 마지막 선형 계층을 제외하고 로딩
# 훈련하지 않을 부분의 파라미터에 영향을 미칠 기울기를 차단(head로 시작하는 파라미터 이름)

In [3]:
# head_linear 외에 지점에 대해 미세 튜닝 훈련을 할 때도 앞 단의 피처 추출 계층이 문제 해법에 최적은 아닐 것이라는 가정하에 head_linear를 랜덤으로 리셋
# 모델 설정에 로딩을 위한 코드 추가
# training.py:124
d = torch.load(self.cli_args.finetune, map_location='cpu')
model_blocks = [
    n for n, subm in model.named_children()
    if len(list(subm.parameters())) > 0 # 파라미터를 가진 최상위 모듈은 제외
]
finetune_blocks = model_blocks[-self.cli_args.finetune_depth:] # 마지막 finetune_depth 블럭 얻기. 미세 튜닝 시 기본값은 1
model.load_state_dict(
    {
        k: v for k,v in d['model_state'].items()
        if k.split('.')[0] not in model_blocks[-1] # 마지막 블럭을 제외시켜 로딩하지 않음. 완전히 초기화된 모델에서 시작하면 앞단의 분류기 출력이 '결절'을 의미하므로 모든 결절 악성으로 판단함
    },
    strict=False, # 없는 것은 제외하면서 모듈의 가중치를 로딩
)
for n, p in model.named_parameters():
    if n.split('.')[0] not in finetune_blocks: # finetune_blocks가 아닌 경우 기울기 필요없음
        p.requires_grad_(False)

In [ ]:
# 아래 명령으로 헤드 부분만 훈련
python3 -m p2ch14.training \
    --malignant \
    --dataset MalignantLunaDataset \
    --finetune data/part2/models/cls_2020-02-06_14.16.55_final-nodule-nonnodule.best.state \
    --epochs 40 \
    --malben-finetune

In [4]:
# 검증셋에 대해 모델 돌려 ROC 곡선 얻자 랜덤인 경우보다 좋지만 기준치 넘지 못하는 원인 확인
# AUC가 천천히 증가하면서 손실값 줄어들고 훈련 손실은 0.3정도에서 능선 이룸
# 나쁜 손실값 곡선은 아니지만 손실값 변화가 정체이므로 다음 세가지 원인 생각할 수 있음
# 1. 결절 여부 분류하는 신경망 훈련에서 얻은 (마지막 컨볼루션의 출력인) 피처가 알고 보니 악성 탐지에는 별 도움 안되는 경우
# 2. 유일하게 훈련하고 있는 부분인 헤드의 용량이 충분하지 않은 경우
# 3. 신경망의 용량이 전반적으로 작은 경우

In [ ]:
# 미세 튜닝 단계에서 완전 연결 훈련으로 충분하지 않다면 미세 튜닝 훈련의 마지막 컨볼루션 블럭 포함시키기
# block4 포함

python3 -m p2ch14.training \
    --malignant \
    --dataset MalignantLunaDataset \
    --finetune data/part2/models/cls_2020-02-06_14.16.55_final-nodule-nonnodule.best.state \
    --finetune-depth 2 \ # 추가
    --epochs 40 \
    --malben-finetune

In [5]:
# 새 모델의 결과 기준치와 비교
# 거짓 양성이 거의 없이 75%의 악성 결절 찾아냄
# 직경 기준치가 주는 65%보다 나은 결과
# 75% 넘어서게 되면 모델 성능은 베이스라인보다 나빠짐
# 분류 문제로 돌아갈 때, ROC 곡선에서 양성과 거짓 양성이 균형을 이룰 만한 지점을 하나 고르기를 원할 것임

In [6]:
# 손실값 곡선을 보면 모델이 매우 일찍 과적합하는 것 확인 가능
# 정규화 메소드 점검해야 함(책에서 안 함)

#### .4 텐서보드에서의 더 많은 출력

In [7]:
# 실행하는 작업을 보기 위해 텐서보드에서 추가해서 보면 도움될 출력 내용
# 텐서보드는 히스토그램 녹화 함수 지원함
# ROC 곡선에 대해서는 지원하지 않으므로 matplotlib 사용해야 함

In [8]:
# 히스토그램
# 악성을 예측한 확률값으로 히스토그램 그릴 수 있음
# 히스토그램으로 모델의 출력에 관해 좀 더 자세하게 살펴보면 출력 확률이 완전히 잘못된 큰 클러스터 있는지 확인 가능

In [ ]:
# metrics_t 텐서에 데이터 넣을 공간 확보
# training.py:31
METRICS_LABEL_NDX=0
METRICS_PRED_NDX=1
METRICS_PRED_P_NDX=2 # 예측 확률 담을 새 인덱스
METRICS_LOSS_NDX=3
METRICS_SIZE = 4

In [9]:
# 이제 레이블, 데이터, 표시할 학습 샘플 수를 담은 global_step 카운터셋을 인자로 writer.add_histogram 호출 가능
# 고정 크기의 bins 집합도 전달
# trainig.py:496
bins = np.linspace(0, 1)

writer.add_histogram(
    'label_neg',
    metrics_t[METRICS_PRED_P_NDX, negLabel_mask],
    self.totalTrainingSamples_count,
    bins=bins
)
writer.add_histogram(
    'label_pos',
    metrics_t[METRICS_PRED_P_NDX, posLabel_mask],
    self.totalTrainingSamples_count,
    bins=bins
)

In [10]:
# 이제 양성 샘플의 예측 분포가 에포크마다 어떻게 변하는지 확인
# 첫 줄의 비결절 양성인 경우에는 신경망이 관찰한 결과가 악성이 아님을 확신하는 의미의 산 모양 발견 가능
# 하지만 한 계층에서 미세 튜닝의 용량 문제 확인 가능
# 왼쪽의 덩어리가 퍼져나가면서 줄어들지 않으며 1.0 부근에서 작게 솟은 지점 존재하며 전체 범위에 대해 확률 질량이 퍼져나가고 있음
# 손실값이 0.3 이하로 줄어들지 않는 모습

In [11]:
# 올바른 영역에서 멀어진 확률 질량이 악성 샘플보다 악성이 아닌 샘플에 대해 훨씬 큰 듯이 보임
# 따라서 신경망은 악성이 아닌 샘플을 악성 샘플보다 자주 잘못 판단하므로 데이터를 다시 밸런싱하여 악성이 아닌 샘플을 더 보여줄 필요 있음
# 이 결론은 훈련에 아무 문제 없다는 전제가 있음
# 일반적으로 훈련을 먼저 고침

In [12]:
# 훈련 부분은 정답에서 매우 날카로운 봉우리를 보여주지만 다른 곳에서는 발견할 수 없음 -> 훈련이 잘 동작함
# 검증에서 가장 두드러진 차이인 악성에 대한 예측 확률이 0인 지점의 작은 봉우리를 볼 수 있음
# 따라서 시스템적인 문제는 악성 샘플을 악성이 아닌 것으로 잘못 분류하는 점
# 두 계층의 미세 튜닝에서 본 과적합

In [ ]:
# ROC를 비롯한 텐서보드의 곡선들
# 헤스토그램 그릴 때 사용한 데이터로 TPR과 FPR 해당하는 tpr과 fpr 각각 계산
# pyplot.figure 추적해 가면서 Summary.Writer 메소드 add_figure에 전달
# training.py:482
fig = pyplot.figure() # 맷플롯립 그림 설정. 통상 아랫줄은 맷플롯립이 하므로 생략 가능
pyplot.plot(fpr, tpr) # 임의의 pyplot 함수
writer.add_figure('roc', fig, self.totalTrainingSamples_count) # 그림을 텐서보드에 추가

In [13]:
# 깊이2 미세 튜닝이 과적합하는 모습과 헤드만 미세 튜닝하면 과적합하지 않는 모습

## 6절 - 진단을 통해 보게되는 것

In [14]:
# 전체 파이프라인 실행할 필요 있음
# 엔드투엔드 진단 스크립트 작성하고 실행

In [15]:
# --malignant-path를 nodule_analysis 호출에 전달하면 이 경로에서 찾은 악성 모델 실행하고 정보 출력함
# 단일 스캔과 --run-validation에 모두 동작하는 옵션

In [16]:
# 엔드투엔트로 결절 85% 탐지하고 악성 중 70% 검출
# 잘못 분류된 결절 관찰
# 애노테이션을 달은 방사선 전문의 의견이 분분한 지점
# 검증셋에서 악성으로 확실하게 판별된 결절을 분리해 놓는 과정 필요

#### .1 훈련셋, 검증셋, 테스트셋

In [19]:
# 검증셋으로 훈련하지 않았지만 검증셋의 모델 성능을 보고 제일 좋은 훈련 에포크를 선택한 방식에 문제 존재
# 일종의 데이터 누출
# 따라서 실제 성능은 조금 떨어질 수 있음

In [20]:
# 이런 이유로 실무에서는 세 가지 데이터셋으로 분류함
# 훈련셋
# 검증셋: 모델 진화 과정에서 가장 잘 동작하는 에포크 정하기 위해 사용
# 테스트셋: 검증셋을 통해 골라낸 최종 모델 성능을 가늠하기 위한, 모델의 입력으로 사용된 적 없는 실세계 데이터

In [21]:
# 훈련 데이터에서 한 덩어리 떼어놔 테스트셋 추가
# 연구되지 않은 분야에서 사용할 시스템 만드는 프로젝트에서는 이 시점에 테스트셋으로 사용할 독립적인 데이터 추가 확보하는 방향 생각해볼 필요 있음

In [22]:
# 우리 모델은 편향될 여지가 다소 있으므로 모든 단계에서 정보 누출 제어하기 위해 노력해야하며 되도록 독립적인 데이터 사용해 정보 누출 여부 검증해야 함

## 7절 - 다음 단계는? 더 많은 데이터와 영감이 필요하다

In [23]:
# 더 이상의 개선은 힘들 것임
# 분류 검증셋의 154개 결절 중 최소 150개를 맞히고 있으모 매 에포크마다 훈련 변경에서 기인하는 약간의 출렁임 있음
# 모델을 개선해도 검증셋에서 나온 결과가 모델 개선으로 인한 것인지 불투명
# 추가적인 개선을 원한다면 검증 스트라이드를 5로 줄이면 검증셋 크기는 두 배가 되고 훈련 데이터의 1/9에 해당함
# 제한적인 훈련 데이터로부터 떼어갈 테스트셋 문제도 해결해야 할 것

In [24]:
# 신경망이 원하는 성능 내지 못할 때 어떤 패턴이 있는지 볼 필요 있지만 프로젝트를 개선할 일반적인 방법 확인

#### .1 과적합 방지: 더 나은 정규화

In [25]:
# 과적합하는 모델을 먼저 만들고 과적합을 줄이는 식의 패턴이 정형화된 사용법

In [26]:
# 전통적인 정규화와 증강
# 8장에서 설명한 정규화 기법 전부 사용하지 않았음
# 드랍아웃 시도해볼만 함
# 비교적 강력한 증강 방법으로 디지털 주름(digital crumple)을 입력에 넣는 탄성 변형(elastic deformation) 있음
# 회전이나 좌우 반전 단독으로 사용하는 경우보다 더 많은 다양성 가할 수 있음

In [27]:
# 더 추상적인 증강
# 크로스엔트로피 손실은 수학적으로 모든 확률 질량을 레이블에 배치하고, 레이블을 원핫 벡터로 나타낼 수 있는 예측과 분포라는 두 확률 분포의 차이를 측정하는 수단
# 신경망에서 과신이 문제가 된다면, 원핫 분포 대신 잘못된 클래스에 작은 확률 질량 배치해보기: 레이블 스무딩(label smoothing)
# 입력과 데이블을 동시에 지저분하게 만드는 믹스업(mixup)
# 손실값에 대한 선형성을 가정하며 랜덤하게 입력과 레이블을 동시에 채우는 방식
# 적절하게 조장한 분포에서 끌어온 가중치로 입력을 조작하는 것
# 실 데이터에서 이런 식으로 혼합된 데이터가 들어오진 않지만 예측 안정성이 높아지며 효과적임

In [30]:
# 최고의 단일 모델을 넘어서: 앙상블링
# 제대로 된 파라미터 알면 모델이 원하는 대로 동작할 것이라고 생각하는 관점일 때 몇 가지 파라미터 조합(몇 가지 모델) 집합을 준비해 각자의 약점 보완하는 방식 선택 가능
# 여러 개의 모델에서 평가하고 출력 합치는 방식을 앙상블링(ensembling)이라 함
# 여러 개의 모델을 훈련하고 모든 모델 실행해 예측을 평균함

In [31]:
# 완전히 독립된 훈련 실행을 사용하거나 모델 구조를 다양하게 만들기도 함
# 단순하게 하려면 한 번의 훈련 실행에서 다양한 모델의 스냅샷 얻을 수도 있음
# 이 스냅샷들을 평균해 사용하는 것이 확률적 가중치 평균(stochastic weight averaging)의 핵심
# 약간의 조정이 필요하지만 하지 않아도 약간의 정확도 향상을 얻을 수 있음

In [33]:
# 신경망이 학습할 내용을 일반화하기
# 평가에 사용하는 출력 외의 다른 출력에 대해서도 모델이 학습하게 하는 멀티태스크 학습(multitask learning) 고려 가능
# 결절 여부 훈련과 악성 여부 훈련을 동시에 진행하는 점에서 밀접하게 관련된 전이 학습과의 차이점

In [34]:
# 추가적인 작업은 없고 레이블되지 않은 부가적인 데이터만 가지고 있다면 준지도 학습(semi-supervised learning) 생각 가능
# 데이터로 모델을 훈련시키고 레이블되지 않은 데이터에 대해 증강되지 않은 샘플을 대상으로 예측
# 증강된 샘플을 대상으로 증강되지 않은 샘플에서와 동일한 예측이 가능하도록 모델 훈련
# 예측이 맞을지 모르지만 데이터의 증강 여부와 상관없이 모델이 일관되게 예측하도록 만드는 것

In [35]:
# 추가적 데이터 없고, 만들기 어려우니 작업(task) 만들기
# 자기 지도 학습(self-supervised learning)의 영역에 속하는 작업으로 프리텍스트 작업(pretext task)라고도 부름
# 대표적 방식으로 입력 일부를 일부러 손상시키는 것
# 그 후 신경망으로 하여금 모델의 많은 부분을 공유하면서 원래 모양을 재구성하도록 훈련시키거나 분류기가 손상된 데이터와 실 데이터 구분할 수 있도록 훈련
# 입력을 손상시키는 방법은 우리가 결정

In [ ]:
# 모델이 학습하는 피처가 꽤 좋아 데이터셋의 다양한 샘플을 구별할 수 있는 경우 대조 학습(contrastive learning) 가능

#### .2 정제된 훈련 데이터

In [36]:
# 훈련 데이터 개선할 방법들
# 악성 분류는 방사선 전문의가 세밀하게 분류한 내용에 기초한 것
# 악성 여부를 이분법으로 나누면서 버린 데이터를 활요하는 쉬운 방법은 다섯 개의 클래스 사용하는 것
# 방사선 전문의의 평가는 좀 더 고르게 나뉜 레이블로 사용할 수도 있음
# 즉 우리는 각각을 원핫 인코딩하고 주어진 결절에 대한 평가에 대해 평균 낼 수 있음
# 이진 분류에서 사용했던 정확도나 ROC, AUC는 사용할 수 없으므로 이런 모델 평가할 새로운 방법 찾아야 함

In [37]:
# 여러 평가 사용하는 또 다른 방법으로 여러 모델 훈련하는 것
# 각 모델은 각가의 방사선 전문의가 달은 애노테이션으로 훈련한 후 추론 단계에서 출력 확률 평균하는 방식으로 모델 앙상블

In [38]:
# 위의 여러 작업 수행에 대해서는 PyLIDC에서 제공하는 애노테이션 데이터로 다시 돌아가 각 애노테이션에 대해 제공되는 다양한 분류 확인 가능하지만 결절에 대해 더 많이 공부해야 함

In [40]:
# 세그멘테이션 면에서는 PyLIDC에서 제공하는 마스크가 더 잘 동작하는지 확인 가능
# LIDC 데이터는 여러 방사선 전문의가 애노테이션을 달아 놓았고 높은 일치도 그룹과 낮은 일치도 그룹으로 나눌 수 있음
# 이를 분류기가 잘 분류하면 쉬움 그룹에 넣고 분류 어려워 전문가가 봐도 모호한 경우에는 어려움 그룹에 넣어 앞의 두 그룹과 일치하는지를 확인하는 것도 흥미로울 수 있음
# 조금 다른 시각으로 문제 접근해 모델 성능의 관점에서 결절 찾아내기 얼마나 어려운지에 대해 쉬움, 중간, 어려움 버킷으로 구분할 수도 있음

In [41]:
# 준비된 데이터 외에 추가적으로 만들어볼 만한 것으로 결절을 악성 유형별로 나눠 놓는 것
# 전문가가 훈련 데이터 보고 암 유형에 따라 결절에 기록해 둔 후, 모델이 해당 유형을 알려주도록 강제하면 더 효과적인 훈련으로 이어질 수 있음

In [ ]:
# 판별이 어려운 경우 역시 전문가에게 제한적으로 반복적인 오차 점검 검토 요구 가능

#### .3 결과 비교와 리서치 논문

In [42]:
# 다른 사람들 프로젝트 확인해 보는 것도 좋음

## 8절 - 결론

In [43]:
# 엔드투엔드 시스템이 CT 스캔 보고 폐암 진단 시도함
# 제품화에 대한 판단은 용도에 따라 다를 것임

In [ ]:
# 놓친 부분은, 제대로 훈련시키고 결과 검증할 때 필요한 엄청난 양의 정리된 데이터셋
# 개별 사례들은 연구 절차에 따라 여러 전문가의 평가 거쳐야하며 일반적인 경우부터 특수 사례까지 다양한 상황을 포함하는 넓은 범위 가지는 적당한 표현 방법 수반되어야 함
# 순수한 연구용부터 임상용 의학적 검증까지 모든 경우에 모델은 확장이 용이한 환경에서 실행할 수 있어야 함

#### .1 커튼 뒤에서

In [44]:
# 잘 꾸며진 장애물과 기회를 만들어 치우친 사례 제공함

In [45]:
# 모든 아이디어가 발견이 되지도 않을 것이며 모든 변경사항이 돌파구가 되지도 않을 것임

In [46]:
# 이미 시도한것, 벽에 부딪힌 것, 잘 동작하지 않은 것, 굳이 유지할 필요 없던 것 정리
# Softmax 대신 HardTanh를 분류 신경망에 사용하기
# HardTanh로 인한 이슈 해결하기 위해 분류 신경망 복잡하게 만들기(스킵 커넥션 등)
# 어설픈 가중치 초기화는 훈련, 특히 세그멘테이션 훈련을 불안정하게 만들었음
# 세그멘테이션 훈련에 전체 CT 단면을 사용한 것
# 세그멘테이션 손실에 SGD로 가중치 준 것. 잘 동작하지 않아 아담 필요했음
# CT 스캔 3차원 세그멘테이션. 우리의 경우는 잘 동작하지 않았지만 딥마인드는 이 방식으로 해냄. 이 방법은 결절 부분 잘라내는 단계 전에 시도했고 당시엔 메모리가 부족했으므로, 현재 설정에 기초해 다시 시도해볼 수 있을 것임
# LUNA 데이터의 class 열의 의미를 잘못 이해해 책 집필 중 내용 다시 작성
# '결과를 빨리 보고 싶은' 꼼수 남겨둿다가 세그멘테이션 모듈이 찾은 결절 후보의 80% 날려 끔찍한 결과 얻었고, 무슨 일인지 파악하느라 시간 소모
# 다양한 옵티마이저, 손실 함수, 모델 아키텍처
# 다양한 훈련 데이터 밸런싱 방법

In [47]:
# 이 책의 많은 부분이 단지 하나의 접근 방법일 뿐 최선이 아님

In [48]:
# 머신러닝에서 주피터 노트북으로 많은 프로그램 작성하는 경우는 흔하지만 주의할 부분도 있음

In [49]:
# precache 사용했던 캐싱 메커니즘 대신, 별도의 전처리 단계 두어 직렬화된 텐서 만들 수도 있음

In [50]:
# 여러 가지 시도하고 최선의 방법 찾는 동시에, 동료와 항상 최대한 유연하게 협업하는 것이 좋음

## 9절 - 연습 문제

In [51]:
# 1. 분류용 테스트셋을 구현하거나 13장 연습 문제의 테스트셋을 재활용하라. 훈련 시 제일 좋은 에포크를 골라내기 위해 검증셋을 사용하되 엔드투엔드 프로젝트를 평가할 때는 테스트셋을 사용하라. 검증셋에 대한 성능이 테스트셋에 대한 성능과 얼마만큼 유사하게 나오는가?

In [52]:
# 2. 비결절, 양성 결절, 악성 결절의 세 가지 분류를 한꺼번에 실행 가능한 모델을 훈련시킬 수 있는가?
#   a. 부류별로 균형이 잡힌 최고의 훈련용 데이터 분할은 어떤 것일까?
#   b. 이 책의 두 단계 접근 방법과 비교하여, 한 번에 수행하는 모델의 성능은 어떤가?

In [53]:
# 3. 분류기를 애노테이션에 대해 훈련시켰으나 세그멘테이션 출력에 대해 수행하고자 한다. 제공된 비결절 대신 훈련에 사용할 비결절 리스트를 세그멘테이션 모델로 만들자.
#   a. 새로운 셋으로 훈련할 때 분류 모델 성능이 개선되는가?
#   b. 새롭게 훈련된 모델에서 어떤 종류의 결절 후보에 가장 큰 변화가 일어나는지 특징을 알 수 있겠는가?

In [54]:
# 4. 패딩된 컨볼루션으로 인해 이미지의 경계 부분에서 전체 이미지를 사용할 때보다 부족한 결과를 얻는다. CT 스캔 단면 경계 근처의 세그멘테이션된 픽셀의 손실을 내부와 비교해서 계산해보라. 둘 간의 측정할 만한 차이가 존재하는가?

In [55]:
# 5. 32*48*48 패치를 사용해 전체 CT에 대한 분류기를 실행해보자. 세그멘테이션 접근 방식과 비교하면 어떤가?